In [1]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = None
pd.options.mode.chained_assignment = None  # default='warn'

## Tồn tạm sản xuất:

					Tồn tạm sản xuất = tồn kho - ( tổng sản xuất cần - tổng đã xuất ) 

					
* tồn kho: Số tồn trong kho hiện tại

* tổng sản xuất cần: Tổng số lượng NPL cần cho lsx đã duyệt + yêu cầu xuất kho + số lượng GCN + yêu cầu xuất đổi trả

* tổng đã xuất: Tổng số lượng NPL đã xuất cho LSX + yêu cầu xuất kho + GCN + yêu cầu xuất đổi trả

	- Xuất kho:
		- assign_product_stockout
		- production_stock_out

	- Nhập kho: 
		- product_stock_in_real_details
		- product_stock_in
		- product_stock_in_reals
		
	- Quản lý LSX: Lên LXS + Đã xuất
	 	- production_order_detail: Đã chạy thao tác xuất bù, hủy NPL, hủy LSX
	 	- production_stock_out
	 	- order_detail

		
	- Gia công ngoài :
		- outsourcing_infos:
		- outsourcing_detail:
		- outsourcing_outsourcing_quota: Xuất NPL đi GCN

	- Yêu cầu đổi trả:
		- return_product_id
		- production_stock_out
		- assign_product_stockout
		
	- Yêu cầu xuất kho: 
		- req_stockout_product_info
		- req_stockout_product_detail
		- production_stock_out
		NOTE: Yêu cầu xuất npl: Không ảnh hưởng đơn hàng, xuất không cần qt :))


	- Note: 	Đã duyệt có tích: Tác nghiệp đã lập định mức
	- Đã duyệt không có tích: Tác nghiệp chưa lập định mức. sale mới xuống đơn hàng
	
	
	- material report:
		- need quantity: số lượng Cần
		- order quantity: số lượng đã đặt
		- need order quantity: số lượng Cần đi gia công cho Thứ cấp
		- in stock: Tồn kho
		- need for outsourcing: cần cho thứ cấp 
		- outsourcing stockout: Xuất cho GCN

In [4]:
dfChild = pd.read_csv("data_csv/Child.csv")
dfParent = pd.read_csv("data_csv/Parent.csv")
dfOrder = pd.read_csv("data_csv/Order.csv")

dfReqStockoutProductDetail = pd.read_csv("data_csv/ReqStockoutProductDetail.csv") # Request Stockout Product Detail
dfOutSourcingDetail = pd.read_csv("data_csv/OutSourcingDetail.csv") # Outsourcing Detail
dfReturnProduct = pd.read_csv("data_csv/ReturnProduct.csv") # Return Product
dfPurchaseOrder = pd.read_csv("data_csv/PurchaseOrder.csv") # Purchase Order
dfStockInPurchaseOrder = pd.read_csv("data_csv/StockInPurchaseOrder.csv") # Purchase Order
dfOutSourcingQuota = pd.read_csv("data_csv/OutSourcingQuota.csv")
# dfOutSourcing = pd.read_csv("OutSourcing.csv")


### Yêu cầu xuất kho

In [5]:
dfReqStockoutProductDetail

,product_id,quantity_req_stockout,exported_req_quantity
0,1,320.25,320.25
1,2,1121.70,1121.70
2,9,1.50,1.50
3,10,6.00,6.00
4,12,11.50,11.50
...,...,...,...
1102,10251,61.00,61.00
1103,10252,21.00,21.00
1104,10253,31.00,31.00
1105,10264,30.00,0.33


In [6]:
dfReqStockoutProductDetail

,product_id,quantity_req_stockout,exported_req_quantity
0,1,320.25,320.25
1,2,1121.70,1121.70
2,9,1.50,1.50
3,10,6.00,6.00
4,12,11.50,11.50
...,...,...,...
1102,10251,61.00,61.00
1103,10252,21.00,21.00
1104,10253,31.00,31.00
1105,10264,30.00,0.33


### <input type="checkbox" disabled checked >  Gia công ngoài 

In [7]:
dfOutSourcingDetail

,product_id,outsourcing_quantity,quantity_stock_in
0,10329,750.0,750.0
1,10328,1400.0,1000.0
2,10326,350.0,NaN
3,10324,208.0,250.0
4,10323,400.0,100.0
...,...,...,...
901,93,4.0,2.1
902,91,52.5,52.5
903,86,19.5,19.5
904,30,13.0,13.0


In [8]:
dfOutSourcingDetail[dfOutSourcingDetail['product_id'] == 10328]

,product_id,outsourcing_quantity,quantity_stock_in
1,10328,1400.0,1000.0


#### Số lượng nguyên phụ liệu cấu thành xuất đi GCN

In [9]:
dfOutSourcingQuota

,product_id,total_quota
0,1,25337.50
1,2,49509.60
2,4,5457.60
3,5,6733.66
4,26,131.00
...,...,...
650,10321,20.04
651,10322,30.24
652,10325,350.00
653,10327,33.76


In [10]:
dfOutSourcingQuota[dfOutSourcingQuota['product_id'] == 10328]

,product_id,total_quota
654,10328,750.0


### Yêu cầu đổi trả

In [11]:
dfReturnProduct

,product_id,return_request_quantity,returned_quantity
0,96,1082.9,990.2
1,3802,25000.0,22850.0
2,3825,16000.0,7315.2
3,4759,2160.0,2160.0
4,5361,1000.0,1000.0
5,7764,220.0,2.6
6,8131,30.0,15.0
7,8170,5000.0,5000.0
8,9028,20.0,20.0
9,9093,220.0,220.0


### Quản lý LSX: Lên LXS + Đã xuất

In [12]:
dfOrder_drop_dup = dfOrder.drop_duplicates(subset=['order_code', 'product_code'], keep='first').reset_index(drop= True)
dfOrder_groupby_sum = dfOrder.groupby(['order_code', 'product_code'], as_index=False)['stockout_quantity'].sum()
dfOrder_groupby_sum.sort_values(by = ['stockout_quantity'])
dfOrder = pd.merge(dfOrder_drop_dup , dfOrder_groupby_sum, on=['order_code', 'product_code'], how='inner').drop(columns = ['stockout_quantity_x']).rename(columns = {"stockout_quantity_y":"stockout_quantity"})
dfOrder

,position,total_quota,parent_product,order_code,product_code,order_quota,production_order_quantity,stockout_quantity
0,Chóp,50.0,10331,PANAMA-NC465-FireBrick-S-PO83499,8-INPL36-464-45-NC,0.10,50.0,25.0
1,Đỉnh,100.0,10330,PANAMA-NC465-FireBrick-S-PO83499,8-INPL35-NAI-DU-NC,0.20,100.0,35.0
2,Chóp,50.0,10330,PANAMA-NC465-DarkMagenta-M-PO83499,8-INPL35-NAI-DU-NC,0.10,50.0,15.0
3,Đỉnh,100.0,10331,PANAMA-NC465-DarkMagenta-M-PO83499,8-INPL36-464-45-NC,0.20,100.0,50.0
4,Đỉnh,100.0,10331,PANAMA-NC465-DarkSlateBlue-L-PO83499,8-INPL36-464-45-NC,0.20,100.0,50.0
...,...,...,...,...,...,...,...,...
23077,Bao nylon,20.0,6731,BF-220330-FCS RED BLUE,9-B03680-WHI-00-HN,0.04,20.0,20.0
23078,Hút ẩm,20.0,6587,BF-220330-FCS RED BLUE,9-HCHOAM-000-00-NP,0.04,20.0,20.0
23079,Thùng,5.0,8225,BF-220330-FCS RED BLUE,9-X44317-Y00-5L-BN,0.01,5.0,5.0
23080,Gai dính bông,45.0,6605,BF-220330-FCS RED BLUE,9-GDTHMB-WHI-20-AN,NaN,NaN,0.0


In [13]:
dfProductionOrder = dfOrder[['parent_product' , 'production_order_quantity', 'stockout_quantity']]
dfProductionOrder.sort_values(by = ['parent_product'])
dfProductionOrderDetail = dfProductionOrder.groupby(['parent_product'], as_index=False)['production_order_quantity' , 'stockout_quantity'].sum()
dfProductionOrderDetail

/tmp/ipykernel_21707/1890485002.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  dfProductionOrderDetail = dfProductionOrder.groupby(['parent_product'], as_index=False)['production_order_quantity' , 'stockout_quantity'].sum()


,parent_product,production_order_quantity,stockout_quantity
0,1,617.4300,509.6400
1,2,294.0100,312.5200
2,9,276.4900,458.7900
3,10,189.4300,375.5000
4,12,201.4415,409.8715
...,...,...,...
2706,10327,0.0000,0.0000
2707,10328,120.0000,0.0000
2708,10329,120.0000,0.0000
2709,10330,400.0000,100.0000


### Tồn tạm sản xuất

In [14]:
df = dfReqStockoutProductDetail.merge(dfOutSourcingDetail, on = 'product_id' , how = 'outer')
df = df.merge(dfReturnProduct, on = 'product_id' , how = 'outer')
df.rename(columns = {'product_id':'parent_product'}, inplace = True)
df = dfProductionOrderDetail.merge(df, on = 'parent_product' , how = 'outer')
df = dfParent.merge(df, on = 'parent_product' , how = 'outer').drop(columns = ['is_outsourcing'])
df = df.fillna(0).reset_index(drop= True)
df

,parent_product,product_code,product_quantity,temporary_quantity,production_order_quantity,stockout_quantity,quantity_req_stockout,exported_req_quantity,outsourcing_quantity,quantity_stock_in,return_request_quantity,returned_quantity
0,1,1-LUOIMA-BLK-80-CM,2882.64,-5568.120000,617.43,509.64,320.25,320.25,0.0,0.0,0.0,0.0
1,2,1-LUOIMA-WHI-80-CM,0.00,-6642.921728,294.01,312.52,1121.70,1121.70,0.0,0.0,0.0,0.0
2,3,1-LUOIMA-LWH-80-CM,0.00,0.000000,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0
3,4,1-LUOIMA-BLK-28-CM,3364.60,7106.441608,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0
4,5,1-LUOIMA-WHI-28-CM,4241.00,6878.663200,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
10183,10328,7-INPL31-466-W2-NC,500.00,680.000000,120.00,0.00,0.00,0.00,1400.0,1000.0,0.0,0.0
10184,10329,7-INPL32-466-WS-EW,750.00,380.000000,120.00,0.00,0.00,0.00,750.0,750.0,0.0,0.0
10185,10330,8-INPL35-NAI-DU-NC,700.00,400.000000,400.00,100.00,0.00,0.00,0.0,0.0,0.0,0.0
10186,10331,8-INPL36-464-45-NC,725.00,600.000000,250.00,125.00,0.00,0.00,0.0,0.0,0.0,0.0


In [15]:
df['total_production_need'] =  df['production_order_quantity'] + df['quantity_req_stockout'] + df['outsourcing_quantity'] + df['return_request_quantity']
df['total_exported'] = df['stockout_quantity'] + df['exported_req_quantity'] + df['quantity_stock_in'] + df['returned_quantity']

In [16]:
df['temporary_production_inventory'] =  df['product_quantity'] - (df['total_production_need'] - df['total_exported'])
dfTemporaryProductionInventory = df
dfTemporaryProductionInventory


,parent_product,product_code,product_quantity,temporary_quantity,production_order_quantity,stockout_quantity,quantity_req_stockout,exported_req_quantity,outsourcing_quantity,quantity_stock_in,return_request_quantity,returned_quantity,total_production_need,total_exported,temporary_production_inventory
0,1,1-LUOIMA-BLK-80-CM,2882.64,-5568.120000,617.43,509.64,320.25,320.25,0.0,0.0,0.0,0.0,937.68,829.89,2774.85
1,2,1-LUOIMA-WHI-80-CM,0.00,-6642.921728,294.01,312.52,1121.70,1121.70,0.0,0.0,0.0,0.0,1415.71,1434.22,18.51
2,3,1-LUOIMA-LWH-80-CM,0.00,0.000000,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00
3,4,1-LUOIMA-BLK-28-CM,3364.60,7106.441608,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,3364.60
4,5,1-LUOIMA-WHI-28-CM,4241.00,6878.663200,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,4241.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10183,10328,7-INPL31-466-W2-NC,500.00,680.000000,120.00,0.00,0.00,0.00,1400.0,1000.0,0.0,0.0,1520.00,1000.00,-20.00
10184,10329,7-INPL32-466-WS-EW,750.00,380.000000,120.00,0.00,0.00,0.00,750.0,750.0,0.0,0.0,870.00,750.00,630.00
10185,10330,8-INPL35-NAI-DU-NC,700.00,400.000000,400.00,100.00,0.00,0.00,0.0,0.0,0.0,0.0,400.00,100.00,400.00
10186,10331,8-INPL36-464-45-NC,725.00,600.000000,250.00,125.00,0.00,0.00,0.0,0.0,0.0,0.0,250.00,125.00,600.00
